# Σημασιολογικός Πυρήνας με Ενσωμάτωση OpenBnB MCP Server

Αυτό το σημειωματάριο δείχνει πώς να χρησιμοποιήσετε τον Σημασιολογικό Πυρήνα με τον πραγματικό OpenBnB MCP server για να αναζητήσετε πραγματικά καταλύματα Airbnb χρησιμοποιώντας το MCPStdioPlugin. Για πρόσβαση στο LLM, χρησιμοποιεί το Azure AI Foundry. Για να ρυθμίσετε τις μεταβλητές περιβάλλοντος, μπορείτε να ακολουθήσετε το [Μάθημα Ρύθμισης](/00-course-setup/README.md)


## Εισαγωγή των Απαραίτητων Πακέτων


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Δημιουργία Σύνδεσης MCP Plugin

Θα συνδεθούμε με τον [OpenBnB MCP server](https://github.com/openbnb-org/mcp-server-airbnb) χρησιμοποιώντας το MCPStdioPlugin. Αυτός ο διακομιστής παρέχει λειτουργικότητα αναζήτησης Airbnb μέσω του πακέτου @openbnb/mcp-server-airbnb.


## Δημιουργία του Πελάτη

Σε αυτό το παράδειγμα, θα χρησιμοποιήσουμε το Azure AI Foundry για την πρόσβαση στο LLM. Βεβαιωθείτε ότι οι μεταβλητές περιβάλλοντος σας έχουν ρυθμιστεί σωστά.


## Ρύθμιση Περιβάλλοντος

Ρυθμίστε τις παραμέτρους του Azure OpenAI. Βεβαιωθείτε ότι έχετε ορίσει τις παρακάτω μεταβλητές περιβάλλοντος:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Κατανόηση της Ενσωμάτωσης OpenBnB MCP

Αυτό το notebook συνδέεται με τον **πραγματικό διακομιστή OpenBnB MCP**, ο οποίος παρέχει λειτουργικότητα αναζήτησης Airbnb.

### Πώς λειτουργεί:

1. **MCPStdioPlugin**: Χρησιμοποιεί επικοινωνία μέσω τυπικής εισόδου/εξόδου με τον διακομιστή MCP
2. **Πραγματικό Πακέτο NPM**: Κατεβάζει και εκτελεί το `@openbnb/mcp-server-airbnb` μέσω npx
3. **Ζωντανά Δεδομένα**: Επιστρέφει πραγματικά δεδομένα ιδιοκτησιών Airbnb από τα APIs τους
4. **Ανακάλυψη Λειτουργιών**: Ο agent ανακαλύπτει αυτόματα τις διαθέσιμες λειτουργίες από τον διακομιστή MCP

### Διαθέσιμες Λειτουργίες:

Ο διακομιστής OpenBnB MCP συνήθως παρέχει:
- **search_listings** - Αναζήτηση ιδιοκτησιών Airbnb ανά τοποθεσία και κριτήρια
- **get_listing_details** - Λήψη λεπτομερών πληροφοριών για συγκεκριμένες ιδιοκτησίες
- **check_availability** - Έλεγχος διαθεσιμότητας για συγκεκριμένες ημερομηνίες
- **get_reviews** - Ανάκτηση κριτικών για ιδιοκτησίες
- **get_host_info** - Λήψη πληροφοριών για τους οικοδεσπότες ιδιοκτησιών

### Προαπαιτούμενα:

- **Node.js** εγκατεστημένο στο σύστημά σας
- **Σύνδεση στο Διαδίκτυο** για τη λήψη του πακέτου διακομιστή MCP
- **NPX** διαθέσιμο (περιλαμβάνεται στο Node.js)

### Δοκιμή της Σύνδεσης:

Μπορείτε να δοκιμάσετε τον διακομιστή MCP χειροκίνητα εκτελώντας:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Αυτό θα κατεβάσει και θα ξεκινήσει τον διακομιστή OpenBnB MCP, στον οποίο στη συνέχεια συνδέεται το Semantic Kernel για πραγματικά δεδομένα Airbnb.


## Εκτέλεση του Agent με τον OpenBnB MCP Server

Τώρα θα εκτελέσουμε τον AI Agent που συνδέεται με τον OpenBnB MCP server για να αναζητήσει πραγματικά καταλύματα Airbnb στη Στοκχόλμη για 2 ενήλικες και 1 παιδί. Μπορείτε να τροποποιήσετε τη λίστα `user_inputs` για να αλλάξετε τα κριτήρια αναζήτησης.


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

### Περίληψη
Συγχαρητήρια! Έχετε δημιουργήσει με επιτυχία έναν AI agent που ενσωματώνεται με αναζήτηση πραγματικών καταλυμάτων χρησιμοποιώντας το Model Context Protocol (MCP):

### Τεχνολογίες που Χρησιμοποιήθηκαν:
- **Semantic Kernel** - Για τη δημιουργία έξυπνων agents με το Azure OpenAI  
- **Azure AI Foundry** - Για δυνατότητες LLM και ολοκλήρωση συνομιλιών  
- **MCP (Model Context Protocol)** - Για τυποποιημένη ενσωμάτωση εργαλείων  
- **OpenBnB MCP Server** - Για πραγματική λειτουργικότητα αναζήτησης Airbnb  
- **Node.js/NPX** - Για την εκτέλεση του εξωτερικού MCP server  

### Τι Έχετε Μάθει:
- **Ενσωμάτωση MCP**: Σύνδεση των Semantic Kernel agents με εξωτερικούς MCP servers  
- **Πρόσβαση σε Δεδομένα σε Πραγματικό Χρόνο**: Αναζήτηση πραγματικών καταλυμάτων Airbnb μέσω live APIs  
- **Επικοινωνία Πρωτοκόλλου**: Χρήση stdio επικοινωνίας μεταξύ του agent και του MCP server  
- **Ανακάλυψη Λειτουργιών**: Αυτόματη ανακάλυψη διαθέσιμων λειτουργιών από MCP servers  
- **Αποκρίσεις Ροής**: Καταγραφή και καταγραφή κλήσεων λειτουργιών σε πραγματικό χρόνο  
- **Απόδοση HTML**: Μορφοποίηση των αποκρίσεων του agent με στυλιζαρισμένους πίνακες και διαδραστικές προβολές  

### Επόμενα Βήματα:
- Ενσωμάτωση επιπλέον MCP servers (καιρός, πτήσεις, εστιατόρια)  
- Δημιουργία συστήματος πολλαπλών agents που συνδυάζει MCP και A2A πρωτόκολλα  
- Δημιουργία προσαρμοσμένων MCP servers για τις δικές σας πηγές δεδομένων  
- Υλοποίηση μνήμης συνομιλίας που διατηρείται μεταξύ συνεδριών  
- Ανάπτυξη του agent σε Azure Functions με ορχήστρωση MCP server  
- Προσθήκη δυνατότητας αυθεντικοποίησης χρηστών και κρατήσεων  

### Κύρια Πλεονεκτήματα της Αρχιτεκτονικής MCP:
- **Τυποποίηση**: Καθολικό πρωτόκολλο για τη σύνδεση AI agents με εξωτερικά εργαλεία  
- **Δεδομένα σε Πραγματικό Χρόνο**: Πρόσβαση σε ζωντανές, ενημερωμένες πληροφορίες από διάφορες υπηρεσίες  
- **Επεκτασιμότητα**: Εύκολη ενσωμάτωση νέων πηγών δεδομένων και εργαλείων  
- **Διαλειτουργικότητα**: Λειτουργεί σε διαφορετικά AI frameworks και πλατφόρμες agents  
- **Διαχωρισμός Αρμοδιοτήτων**: Σαφής διάκριση μεταξύ της λογικής του AI και της πρόσβασης σε εξωτερικά δεδομένα  



---

**Αποποίηση ευθύνης**:  
Αυτό το έγγραφο έχει μεταφραστεί χρησιμοποιώντας την υπηρεσία αυτόματης μετάφρασης [Co-op Translator](https://github.com/Azure/co-op-translator). Παρόλο που καταβάλλουμε προσπάθειες για ακρίβεια, παρακαλούμε να έχετε υπόψη ότι οι αυτοματοποιημένες μεταφράσεις ενδέχεται να περιέχουν σφάλματα ή ανακρίβειες. Το πρωτότυπο έγγραφο στη μητρική του γλώσσα θα πρέπει να θεωρείται η αυθεντική πηγή. Για κρίσιμες πληροφορίες, συνιστάται επαγγελματική ανθρώπινη μετάφραση. Δεν φέρουμε ευθύνη για τυχόν παρεξηγήσεις ή εσφαλμένες ερμηνείες που προκύπτουν από τη χρήση αυτής της μετάφρασης.
